<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> Tarea 4</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> 2024-2</h2>
</center>

# Consultas

* Enviar a: **tareas.inf285@gmail.com**
* Se recibirán consultas en **tres** bloques:
1. El primer bloque será desde las 12:15 hrs. hasta las 17:00 hrs. del día jueves, donde todas las preguntas recibidas dentro de este bloque se responderán con seguridad a partir de las 17:01 hrs. del día jueves.
2. El segundo bloque de consultas será desde las 17:01 hrs. del día jueves hasta las 08:15 hrs. del día viernes, donde las preguntas recibidas dentro de este bloque horario se responderán con seguridad a partir de las 08:16 hrs. del día viernes.
3. El tercer, y último bloque, de consultas será desde las 08:16 hrs. hasta las 14:30 hrs. del día viernes, donde las preguntas recibidas dentro de este bloque horario se responderán con seguridad a partir de las 14:31 hrs. del día viernes.

# Contexto

La ecuación de difusión de calor en una dimensión describe las calorías $u(x,t)$ en un punto $x \in [0,L]$ de una varilla en un determinado tiempo $t$. Considerando que el calor de la varilla se perderá con el tiempo podemos definir $M(t)$ como la **cantidad total de calorías** en la varilla en un determinado tiempo $t$


$$
M(t) = \int_{0}^{L}u(x,t)^2dx
$$

Al trabajar esta ecuación obtenemos:

$$
u_t = ku_{xx}
$$

donde k es el factor de transferencia de calor de la varilla que se considerará de valor constante.

# Librerías permitidas

In [254]:
import numpy as np
from scipy.optimize import bisect

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from ipywidgets import IntSlider
from ipywidgets import interact

from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# Pregunta 1

Se sabe que la distribución inicial de calor $u(x,0)$ tiene la forma:

$$
u(x,0) = Ce^{-\alpha x}sin(βx)+ln(1+x),
$$

donde los valores de $\alpha$ y $\beta$ son conocidos. No así la constante $C$.

Tambien se conoce la cantidad total de calorías en el tiempo 0:

$$
\int_{0}^{L} (\textit{C} e^{-\alpha x} \sin(\beta x) + ln(1+x))^2 dx = M_0
$$

Por lo tanto, podemos utilizar métodos ya conocidos para encontrar el valor de la constante $C$.

In [255]:
# no modificar
L = 1.0
M0 = 2.0
alpha = 2.0
beta = np.pi

1.1 **(10pts)**
Defina la funcion $u0(x, C)$ que se utilizará exclusivamente para calcular el valor de la constante $C$ desconocida. (Esta $u_0$ es distinta de $u_t$, ya que $u_0$  considera un tiempo $t=0$. Dicho de otro modo, $u_t(x,0)=u_0$).

In [256]:
def u0(x, C):
    """
    input:
    x: (ndarray) positions to evaluate
    C: (float) constant
    output:
    (float) initial heat distribution
    """
    # acá va su código
    #--------------------------

    # usamos la formula entregada directamente
    u0_function = C * np.exp(-alpha * x) * np.sin(beta * x) + np.log(1 + x)

    #--------------------------
    return u0_function

1.2 **(10pts)** Utilizando la definición de $u0$, calcule el valor de la constante $C$.

*Hint 1: f(x) = c, can be reorganized to: g(x) = f(x) - c = 0.*

*Hint 2: g(x)=0 is a well known problem.*

In [257]:
def trapezoidal_integral(f, x, C):
    """
    input:
    f (function)          : function to integrate
    x (ndarray)           : positions to evaluate
    C (float)             : constant

    output:
    aproximation (float)  : aproximation of the integral
    """
    # acá va su código
    #--------------------------

    # primero vectorizamos la function
    f_vec = np.vectorize(f)

    # la funcion evaluada en los puntos
    y = f_vec(x, C)**2

    # la diferencia entre los x a evaluar
    dx = np.diff(x)
     
    # el promedio entre los valores evaluados
    y_avg = (y[:-1] + y[1:]) / 2

    # con este metodo, la integral se calcula
    # con el promedio entre los puntos
    aproximation = np.sum(dx * y_avg)
    
    #--------------------------
    return aproximation

In [258]:
def find_C(M0, x):
    """
    input:
    M0 (float)  : initial energy
    x (ndarray) : positions to evaluate

    output:
    C (float)   : constant that satisfies the energy condition
    """
    # acá va su código
    #--------------------------

    # definimos una cantidad maxima de iteraciones
    iterations = 10

    # obtenemos C en funcion inversa de u0
    C_f_inv = lambda C: trapezoidal_integral(u0, x, C) - M0

    # obtenemos el valor de C usando biseccion
    C = bisect(C_f_inv, 0, iterations)

    print(f"El valor de C es: {C}")
    
    #--------------------------
    return C

In [259]:
# Ejecute
N = 1000
x = np.linspace(0, L, N)
C = find_C(M0, x)
u_0 = lambda x: u0(x, C)

El valor de C es: 3.683508788957397


# Pregunta 2

2.1 **(10 pts)**
Realice la discretización de la función $u_t = ku_{xx}$ para obtener la progresión de la solución de la ecuación diferencial:



**Respuesta:**

### Aproximación por diferencias finitas
Derivadas espaciales ($u_{xx}$​):

Usa una aproximación centrada para la segunda derivada:

$$ u_{xx} \approx \frac{u_{i−1} − 2u_{i} + u_{i+1}} {\Delta x^{2} } $$

donde $u_{i}$​ representa el valor de $u(x,f)$ en el punto $x_{i}$

Con esto podemos usar el Metodo de _Forward-Euler_:

$$ u_{t} \approx \frac{u_{i}^{n + 1} − u_{i}^{n}}{\Delta t} $$

Combinando con $u_{xx}$ tenemos:

$$ u_{i}^{n+1} = u_{i}^{n} + \frac{kΔt}{\Delta x^{2}}(u_{i−1}^{n} - 2u_{i}^{n} + u_{i+1}^{n}) $$

2.2 **(15 pts)** Aplique métodos numéricos para resolver esta ecuación para un tiempo $t\in [0,T]$. Tome en cuenta las siguientes consideraciones:

*   $x \in [0,L]$
*   $u(x,0) = ux0(x)$
*   $u(0,t) = u0t(t)$
*   $u(L,t) = uLt(t)$


In [260]:
#Función entregada
def spatial_derivative(u, dx):
  du2 = np.zeros_like(u)
  du2[1:-1] = (u[2:] -2*u[1:-1] + u[:-2])/ dx**2
  return du2


In [278]:
def heat_diffusion(T, L, ux0, u0t, uLt, k, N_x = 100, N_t=100):
    """
    input:
    T       : (int) Max time of simulation
    L       : (int) length of the rod
    ux0     : (callable) function that describes u(x,0)
    u0t     : (callable) function that describes u(0,t)
    uLt     : (callable) function that describes u(L,t)
    k       : (int) diffusion coeficient
    N_x     : (int) number of steps for x
    N_t     : (int) number of steps for T

    output:
    u       : (2D array) matrix with the values values u(x,t)
    """
    # acá va su código
    #--------------------------
    
    # discretizamos la variable espacial
    dx = L / (N_x - 1)
    x = np.linspace(0, L, N_x)

    # discretizamos la variable temporal
    dt = T / (N_t - 1)
    t = np.linspace(0, T, N_t)

    # parametro de estabilidad
    r = k * dt / dx**2

    # creamos la matriz vacia para 
    # llenarla connumoers
    u = np.zeros((N_t, N_x))

    # establecemos las condiciones de frontera
    u[0, :] = ux0(x)
    u[:, 0] = u0t(t)
    u[:, -1] = uLt(t)
    
    # iteramos sibre los valores para obtener el siguiente
    for n in np.arange(0, N_t - 1):
        u[n+1, 1:-1] = u[n, 1:-1] + k * dt * spatial_derivative(u[n, :], dx)[1:-1]

    #--------------------------
    return(u)

In [279]:
# No modifique, solo ejecute
L = 10
T = 1
k = 0.05
N_x = 100
N_t = 100
u0t = lambda x: 0
uLt = lambda x: 0

u = heat_diffusion(T,L,u_0,u0t,uLt,k)

In [280]:
# No modifique, solo ejecute
def plot_interactive(elev=40,azim=230):
    # Crear la figura para el gráfico 3D
    x = np.linspace(0,10,100)
    t = np.linspace(0,1,100)
    X_l,T_l = np.meshgrid(x,t)
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Graficar la superficie
    surf = ax.plot_surface(X_l, T_l, u, cmap="hot", linewidth=0, antialiased=False)

    ax.view_init(elev,azim)

    # Etiquetas de los ejes
    ax.set_xlabel("Espacio (x)")
    ax.set_ylabel("Tiempo (t)")
    ax.set_zlabel("Temperatura u(x,t)")
    ax.set_title("Evolución de la ecuación de calor en 3D")

    # Mostrar el gráfico
    plt.show()

elev_widget = IntSlider(min=0, max=180, step=10, value=40)
azim_widget = IntSlider(min=0, max=360, step=10, value=230)
interact(plot_interactive, elev=elev_widget, azim=azim_widget)


interactive(children=(IntSlider(value=40, description='elev', max=180, step=10), IntSlider(value=230, descript…

<function __main__.plot_interactive(elev=40, azim=230)>

### ¡Si todo ha salido bien el gráfico debería verse así!
<img src="https://media.discordapp.net/attachments/1170178625023246348/1308608276950421514/image.png?ex=673e8fed&is=673d3e6d&hm=2d6f021cbd7568ae15ddb78e2e7ce52fd513d7dcabf268632830b7f11c44ea32&=&format=webp&quality=lossless" style="width:20%">

Otra prueba que puede realizar es utilizando un $u_0(x) = \frac{sin(x*\pi)^2}{2}$

In [264]:
L = 10
T=1
k = 0.05
N_x = 100
N_t = 100
u_0 = lambda x: (np.sin(x*np.pi)**2)/2
u0t = lambda x: (np.log(x+3)*(x+3))**-2
uLt = lambda x: x/200

u = heat_diffusion(T,L,u_0,u0t,uLt,k)
elev_widget = IntSlider(min=0, max=180, step=10, value=40)
azim_widget = IntSlider(min=0, max=360, step=10, value=230)
interact(plot_interactive, elev=elev_widget, azim=azim_widget)

interactive(children=(IntSlider(value=40, description='elev', max=180, step=10), IntSlider(value=230, descript…

<function __main__.plot_interactive(elev=40, azim=230)>

### Este es el resultado esperado:

<img src="https://media.discordapp.net/attachments/1170178625023246348/1308625356734988359/image.png?ex=673e9fd6&is=673d4e56&hm=bc516c72b8d492cd18d78b309b206c6edbb61f6fcc246c5c4cdcd0f71deff725&=&format=webp&quality=lossless" style="width:20%">

2.3 **(15 pts)** Realice un análisis de esta ecuación y su solución ¿Cómo se comporta el calor en la varilla con el paso del tiempo? ¿Cómo afecta las 3 funciones $ux0$, $u0t$ y $uLt$ al comportamiento del calor en la varilla?
Si desea experimentar con la celda de arriba, siéntase libre de hacerlo y compartir sus descubrimientos con una foto 😄

**Respuesta:**

El calor en la varilla se distribuye con el tiempo debido a la conducción térmica, tendiendo a igualarse. 

La temperatura inicial está dada por $ux0$, mientras que $u0t$ y $uLt$ controlan cómo evoluciona el calor en los extremos. 

Si estas condiciones son constantes, la temperatura en los extremos no cambia; si varían, afectan toda la distribución de calor en la varilla.

# Pregunta 3
Ahora expandiremos nuestra ecuación de difusión térmica de una dimensión a dos dimensiones, es decir, ahora contamos con 3 variables para determinar el calor en un punto de la placa.

La ecuación diferencial parcial $(PDE)$ a resolver es:

$$
\frac{\partial U}{\partial t} = k \left( \frac{\partial^2 U}{\partial x^2} + \frac{\partial^2 U}{\partial y^2} \right)
$$

donde:



*   **$U(x, y, t)$**: define el calor en la posición **$(x, y)$** y tiempo **$t$**.


Consideraciones:

* $x\in[0,L_x]$
* $y\in[0,L_y]$

* $U(x,y,0) = f(x,y)$
* $U(0,0,t) = T1$
* $U(L,0,t) = T2$
* $U(0,L,t) = T3$
* $U(L,L,t) = T4$

**T1, T2, T3 y T4 son constantes.**

3.1 **(10 pts)** Discretice la ecuación usando diferencias finitas para obtener el Laplaciano y defina la evolución temporal del sistema:

**Respuesta:**

Para discretizar la ecuación de difusión térmica en dos dimensiones, utilizamos diferencias finitas. La derivada temporal se puede aproximar como:

$$\frac{U(x,y,t+\Delta t) - U(x,y,t)}{\Delta t} = 
k \left( \frac{U(x+\Delta x,y,t) - 2U(x,y,t) + U(x-\Delta x,y,t)}{\Delta x^2} + \frac{U(x,y+\Delta y,t) - 2U(x,y,t) + U(x,y-\Delta y,t)}{\Delta y^2} \right)$$

Reorganizando para obtener $U(x,y,t+\Delta t)$:

$$
U(x,y,t+\Delta t) = U(x,y,t) + k \Delta t \left( \frac{U(x+\Delta x,y,t) - 2U(x,y,t) + U(x-\Delta x,y,t)}{\Delta x^2} + \frac{U(x,y+\Delta y,t) - 2U(x,y,t) + U(x,y-\Delta y,t)}{\Delta y^2} \right)
$$

Donde:
- $\Delta t$ es el paso de tiempo.
- $\Delta x$ y $\Delta y$ son los pasos espaciales en las direcciones $x$ e $y$ respectivamente.
- $k$ es el coeficiente de difusión térmica.

Esta fórmula nos permite calcular la evolución de la temperatura en la placa a lo largo del tiempo.


3.2.1 **(5 pts)** Implemente la funcion $Uxy0$ usando el siguiente perfil
Gaussiano:

$$
U(x, y, 0) = U_{\text{max}} \cdot \exp \left( -\frac{(x - \frac{L}{2})^2 + (y - \frac{L}{2})^2}{c^2} \right)
$$


In [277]:
def Uxy0(x,y):
  """
  input:
  x      :(int) x coordinate of a pint in the plank
  y      :(int) y coordinate of a pint in the plank

  output:
  U      :(float) value of the heat in a point of the plank in t = 0
  """
  L = 0.1
  c = 0.04
  Umax = 20
  # acá va su código
  #--------------------------

  # implementado directamente igual que en la primera parte
  U = Umax * np.exp(-(np.power((x - L / 2), 2) + np.power((y - L / 2), 2)) / np.power(c, 2))
  
  #--------------------------
  return U

3.2.2 **(5pts)** ¿Cómo se comporta este perfil Gaussiano en la plancha? Justifique e identifique los componentes del perfil

**Respuesta:**

3.3 **(20 pts)** Implemente diferencias finitas para resolver el sistema de ecuaciones:



In [275]:
def heat_diffusion_2d(Lx, Ly, k, T0, T1, T2, T3, T4, nx, ny, dt, tf):
    """
    input:
    Lx, Ly          : (int) dimensiones del dominio en metros (longitud en x y en y)
    k               : (int) difusividad térmica (m^2/s)
    T0              : temperatura inicial del campo interno
    T1, T2, T3, T4  : condiciones de frontera (en x=0, x=Lx, y=0, y=Ly)
    nx, ny          : número de puntos en las direcciones x e y
    dt              : paso de tiempo (s)
    tf              : tiempo final (s)

    output:
    T               : arreglo de temperaturas para cada tiempo
    """
    # acá va su código
    #--------------------------

    # igual que antes, calculamos los pasos
    dx = Lx / (nx - 1)
    dy = Ly / (ny - 1)

    # lo mismo pero para el tiempo
    nt = int(tf / dt) + 1

    # creamos una matriz de temperaturas para cada tiempo
    T = np.zeros((nx, ny, nt))

    # establecemos las condiciones iniciales
    T[:, :, 0] = T0
    T[0, :, :] = T1
    T[-1, :, :] = T2
    T[:, 0, :] = T3
    T[:, -1, :] = T4
    
    # iteramos para obtener el valor siguiente
    for n in range(0, nt - 1):  
        T[1:-1, 1:-1, n+1] = T[1:-1, 1:-1, n] + k * dt * (
            spatial_derivative(T[:, :, n], dx) +
            spatial_derivative(T[:, :, n].T, dy)
        ).T[1:-1, 1:-1]

    #--------------------------

    return T

In [276]:
#No modifique solo ejecute

Lx = 0.1  # longitud del lado x (m)
Ly = 0.1  # longitud del lado y (m)
k = 1.172E-5  # Difusividad térmica (m^2/s) para acero, 1% carbono
T1 = 0  # frontera inferior
T2 = 0  # frontera superior
T3 = 0  # frontera izquierda
T4 = 0  # frontera derecha
nx = 40  # número de puntos en x
ny = 40  # número de puntos en y
dt = 0.1  # paso de tiempo (s)
tf = 10   # tiempo final (s)

T0 = np.zeros((nx, ny))
for i in range(nx):
    for j in range(ny):
        T0[i, j] = Uxy0(i*(Lx/(nx-1)),j*(Ly/(ny-1)))



def see_results(elev=40, azim=230, Tiempo=0):
    T = heat_diffusion_2d(Lx, Ly, k, T0, T1, T2, T3, T4, nx, ny, dt, tf)
    X = np.linspace(0, Lx, nx, endpoint=True)
    Y = np.linspace(0, Ly, ny, endpoint=True)
    X, Y = np.meshgrid(X, Y)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    ax.view_init(elev,azim)

    surf = ax.plot_surface(X, Y, T[:, :, Tiempo], cmap='gist_rainbow_r', edgecolor='none')
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_zlabel('T [°C]')
    plt.show()


T_widget =  IntSlider(min=0, max=100, step=1, value=0)
elev_widget = IntSlider(min=0, max=180, step=10, value=40)
azim_widget = IntSlider(min=0, max=360, step=10, value=230)
interact(see_results, elev=elev_widget, azim=azim_widget, Tiempo=T_widget)

interactive(children=(IntSlider(value=40, description='elev', max=180, step=10), IntSlider(value=230, descript…

<function __main__.see_results(elev=40, azim=230, Tiempo=0)>

### Si hiciste todo bien, tu gráfico debería verse así:

<img src="https://media.discordapp.net/attachments/1170178625023246348/1308656987776483348/image.png?ex=673ebd4b&is=673d6bcb&hm=dfe12ffb107716848d9d5f858e416c12f0a205d8bff469be9cac24021429fcca&=&format=webp&quality=lossless" style="width:20%">



**¡Ya ha llegado al final de la tarea 4! Ahora debe enviarla antes de las 23:55 hrs. del viernes 22 de noviembre!**  

**Recuerde que el archivo de entrega debe seguir el siguiente formato: “apellido1_apellido2_tarea_numero.ipynb” en minúsculas, y no olvide revisar el reglamento de tareas.**